In [1]:
import pandas as pd
import skopt

In [2]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
dataframe = pd.read_csv(url, header=None)
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(351, 34) (351,)


In [3]:
# define the space of hyperparameters to search
from skopt.space import Real, Categorical, Integer
search_space = list()
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [4]:
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
import numpy as np

@use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    model = SVC()
    model.set_params(**params)
    print(params)
    # define test harness
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # calculate 5-fold cross validation
    result = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring='accuracy')
    estimate = np.mean(result)
    # convert from a maximizing score to a minimizing score
    return 1.0 - estimate



In [5]:
result = gp_minimize(evaluate_model, search_space)

{'C': 12.351293205610316, 'kernel': 'rbf', 'degree': 2, 'gamma': 0.003405157503869575}
{'C': 0.06067218187426075, 'kernel': 'rbf', 'degree': 4, 'gamma': 0.00029918444981328804}
{'C': 0.0033080247089509293, 'kernel': 'poly', 'degree': 1, 'gamma': 0.0007629783995015235}
{'C': 0.00023739558720687636, 'kernel': 'linear', 'degree': 5, 'gamma': 2.5522337930488416e-06}
{'C': 5.531062173415998e-05, 'kernel': 'rbf', 'degree': 3, 'gamma': 2.4441020508277914e-06}
{'C': 13.124636720745386, 'kernel': 'rbf', 'degree': 3, 'gamma': 0.06533213007097796}
{'C': 0.0045089407516935385, 'kernel': 'linear', 'degree': 3, 'gamma': 0.020820881750815316}
{'C': 0.5173453572898051, 'kernel': 'rbf', 'degree': 3, 'gamma': 5.322606103572869e-05}
{'C': 8.68044150421356e-05, 'kernel': 'poly', 'degree': 2, 'gamma': 0.0004551953343151619}
{'C': 0.0049263015338848635, 'kernel': 'rbf', 'degree': 5, 'gamma': 0.0930278544656824}
{'C': 99.50502591187268, 'kernel': 'rbf', 'degree': 3, 'gamma': 5.1188050470184105}
{'C': 12.4437

In [6]:
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

Best Accuracy: 0.953
Best Parameters: [9.74508348467212, 'rbf', 3, 0.05699476534837755]
